<a href="https://colab.research.google.com/github/waltermartemucci/CLPBgroup17/blob/main/Exercise_4/ex3q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#creating deepcopy of model instances
from copy import deepcopy

#array manipulations
import numpy as np

#dataframe and its manipulations
import pandas as pd

#plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

#selected plotting functions
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

#classes for grid search and cross-validation, function for splitting data and evaluating models
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,confusion_matrix,roc_curve

#XGBoost library
import xgboost as xgb

#Python standard libraries
import time
import warnings

#setting default resolution of figures
mpl.rcParams['figure.dpi'] = 200

from sklearn.model_selection import train_test_split


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


ModuleNotFoundError: ignored

In [ ]:
pip install tsfresh

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 14
from tsfresh import extract_features

from xgboost import XGBClassifier, plot_tree

%run /content/plot.py

In [ ]:
#read data

str0 = 'ts_L60_Z12_A500_DX50_bias5_N10000.dat'
fnamex='DATA/x_'+str0
fnamey='DATA/y_'+str0

x = np.loadtxt(fnamex, delimiter=" ",dtype=float)
N,L = len(x), len(x[0])

Show_data(x,L,"original data")

# note: here it does not need to be converted to the 3-bit version, a label remains y[i]=0,1,2
y = np.loadtxt(fnamey, dtype=int)
n_class = 3    #  = len(np.unique(y))
print('data: ',N)

In [ ]:
#load data into dataframe
df_x = pd.read_csv('DATA/x_ts_L60_Z12_A500_DX50_bias5_N10000.dat',
                         header=None, sep='\s\s+', engine='python')
df_y = pd.read_csv('DATA/y_ts_L60_Z12_A500_DX50_bias5_N10000.dat',
                         header=None, sep='\s\s+', engine='python')
#dropping id columns
df_x.drop(df_x.columns[-1], axis=1, inplace=True)
df_y.drop(df_y.columns[-1], axis=1, inplace=True)
#checking x and data types
df_x.info()
df_y.info()

In [ ]:
#replace non interpretable values with zeroes
df_x.loc[df_x[lambda x: x == '?'].any(axis=1), df_x[lambda x: x == '?'].any(axis=0)]
df_x.replace(to_replace='?', value=0, inplace=True)



In [ ]:
#checking autocorrelations, whether data are time series
f, axes = plt.subplots(29,2,figsize=(18,180))
for i,ax in enumerate(axes):
    plot_acf(df_train.iloc[:,i], zero=False, ax=ax[0])
    plot_pacf(df_train.iloc[:,i], zero=False, ax=ax[1])
plt.show()

In [ ]:
#split train and test
#train test split with randomization performed (although randomization is not necessary)
X_train, X_test, y_train, y_test = train_test_split(df_train.iloc[:,1:], df_train.iloc[:,:1], test_size=0.2, random_state=122, stratify=df_train.iloc[:,:1])